# Работа со строковыми значениями

Материалы:
* Макрушин С.В. Лекция 8: Работа со строковыми значениям
* https://pyformat.info/
* https://docs.python.org/3/library/re.html
* https://tproger.ru/translations/regular-expression-python/
* https://realpython.com/nltk-nlp-python/

## Задачи для совместного разбора

1. Вывести на экран данные из словаря `obj` построчно в виде `k = v`, задав формат таким образом, чтобы знак равенства оказался на одной и той же позиции во всех строках. Строковые литералы обернуть в кавычки.

In [ ]:
obj = {  
         'home_page': 'https://github.com/pypa/sampleproject',
         'keywords': 'sample setuptools development',
         'license': 'MIT'
      }

2. Дана строка 'aaa--bbb==ccc__ddd'. Написать регулярное выражение для разбивки строки на список ['aaa','bbb','ccc','ddd'].

3. Проверить корректность введенного E-mail

4. Разбейте текст формулировки задачи 1 на слова.

## Лабораторная работа 8

### Форматирование строк

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


1.1 Загрузите данные из файла `recipes_sample_with_tags_ingredients.csv` (__ЛР5__) в таблицу `recipes` При помощи форматирования строк выведите информацию об id рецепта и кол-ве игредиентов 5 случайных рецептов в виде таблицы следующего вида:

    
    |    id     | n_in  |
    |-------------------|
    |  400894   |  13   |
    |   68588   |   8   |
    |  362081   |   6   |
    |   53408   |  12   |
    |  221203   |   4   |

In [ ]:
import pandas as pd
recipes = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Drafts/recipes_sample_with_filled_nsteps.csv")
new_recipes = recipes.sample(n=5)
print (new_recipes)
#print("|{0:^11}| {1:^6}|\n|{2}|".format("id", "n_in","-"*19))
for index, row in new_recipes.iterrows():
    print("|{0:^11}| {1:^6}|".format(row['id'], row['n_ingredients'])) 

1.2 Напишите функцию `show_info`, которая для рецепта по его `id` создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название"

1. Шаг 1.
2. Шаг 2.
----------
#тэг1 #тэг2
```

    
Данные для создания строки получите из файлов `recipes_sample_with_tags_ingredients.csv`, `steps_sample.xml` (__ЛР4__) и `tags_sample.csv` (__ЛР5__). 
Выведите созданную строку на экран.

In [ ]:
pip install pydantic


In [ ]:
from bs4 import BeautifulSoup
from typing import List
from pydantic import validate_arguments, ValidationError



class Recipe:
    @validate_arguments
    def __init__(self, recipe_id: int, name: str) -> None:
        self._recipe_id = recipe_id
        self._name = name
        self._steps_list = []
        self._tags_list = []
    
    @validate_arguments
    def add_tag(self, data: str) -> None:
        tag = f"#{data}"
        if tag not in self._tags_list:
            self._tags_list.append(tag)
    
    @validate_arguments
    def add_step(self, data: str) -> None:
        if data not in self._steps_list:
            self._steps_list.append(data)    
    
    def show_info(self) -> None:
        print(self.__repr__())
        
    def __repr__(self) -> str:
        steps_str = "\n".join(map(lambda x: f"{x[0]}. {x[1].capitalize()}.",enumerate(self._steps_list, 1)))
        tags_str = " ".join(self._tags_list)
        final_str = f"\"{self._name.capitalize()}\"\n{steps_str}\n{'-'*10}\n{tags_str}"
        return final_str



class RecipeManager:
    def __init__(self, names: pd.core.frame.DataFrame, steps: BeautifulSoup, tags: pd.core.frame.DataFrame) -> None:
        self._names = names
        self._steps = steps
        self._tags = tags
        
        self.recipes = {}
        
        self.names_processing()
        self.steps_processing()
        self.tags_processing()
    
    def names_processing(self) -> None:
        for item in self._names.apply(lambda x: (x["id"], x["name"]), axis=1):
            r_id, r_name = item
            self.recipes[r_id] = Recipe(r_id, r_name)
        
    def steps_processing(self) -> None:        
        for recipe in self._steps.find_all('recipe'):
            recipe_id = int(recipe.find("id").get_text())
            for step in recipe.find_all("step"):
                step_text = step.get_text()
                self.recipes[recipe_id].add_step(step_text)
                
    def tags_processing(self) -> None:
        for item in self._tags.apply(lambda x: (x["id"], x["tag"]), axis=1):
            recipe_id, tag = item
            self.recipes[recipe_id].add_tag(tag)    
    
    def find(self, recipe_id: int) -> Recipe:
        if recipe_id not in self.recipes:
            raise ValueError(f"No recipe with this id -> {recipe_id} ")
        return self.recipes[recipe_id]

    def ids(self) -> List[int]:
        return list(self.recipes.keys())

content = open("/content/drive/MyDrive/Colab Notebooks/lab 4/steps_sample.xml","r").read()
steps = BeautifulSoup(content,'xml')

tags = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/lab 5/tags_sample.csv")
recipe_manager = RecipeManager(recipes, steps, tags)



recipe_manager.find(44123)

## Работа с регулярными выражениями

В задачах данного блока подразумевается, что вы не будете использовать никаких строковые методы (`split`, `startswith` и т.д.). Все задачи необходимо решить при помощи регулярных выражений.

2.1 Посчитайте кол-во отзывов, в которых встречаются числа.

In [13]:
import re
reviews = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/lab 3/reviews_sample.csv")
reviews_list = [item for item in reviews["review"].to_list() if isinstance(item, str)]

pattern = re.compile(r'.*[0-9].*')
sum([pattern.match(item) is not None for item in reviews_list])

47336

2.2 Найдите все смайлики в отзывах. Смайлик состоит из трех частей: глаза (символ `=` или `:`), нос (символ `-`), губы (символ `)` или `(`). Смайлик может иметь вид "глаза-нос-губы" или "губы-нос-глаза". Нос может отсутствовать.

In [ ]:
pattern = re.compile(r'([()]-?[=:])|([=:]-?[()])')
results_set ={item for item in reviews_list if pattern.findall(item) is not None}
[print(f"{x}\n{'-'*100}\n") for x in results_set]

2.3 Проверьте, что все даты в датасете имеют вид "YYYY-MM-DD". Продемонстрируйте работу вашего решения, приведя пример из датасета и контрпример не из датасета.

In [16]:
pattern = re.compile(r'^[0-9]{4}\-[0-9]{2}\-[0-9]{2}$')
result1 = sum([pattern.match(item) is not None for item in reviews["date"]])
result2 = reviews["date"].shape[0]
assert result1 == result2
for test_str in ("1901-01-01p","1901-101-01","9999.00.00"):
    assert pattern.match(test_str) is None

2.4 Используя строку-результат задачи 1.2, найдите первое слово каждого шага в рецепте

In [ ]:
import random

pattern = re.compile(r'[0-9]+\.\ ([A-z]+)')
recipe_id = random.choice(recipe_manager.ids())
current_str = recipe_manager.find(recipe_id).__repr__()
print(current_str)

pattern.findall(current_str)

2.5 Используя регулярные выражения, удалите из описаний все символы, кроме английских букв, цифр и пробелов. Сохраните предобработанные описания в файл `preprocessed_descriptions.csv`, содержащий 2 столбца: `name` и `preprocessed_descriptions`.

In [ ]:
column2process = "preprocessed_descriptions"
pattern = re.compile(r'[^A-Za-z0-9 ]')

def filler(x):
    result = pattern.sub('', x[column2process])
    return result

descriptions = recipes["name"].to_frame().join(recipes["description"].fillna(""))
descriptions = descriptions.rename(columns={"description": column2process})
descriptions[column2process] = descriptions.apply(lambda x: filler(x), axis=1)
print (descriptions)
descriptions.to_csv("preprocessed_descriptions.csv")

### Сегментация текста

3.1 Разбейте отзывы из файла `recipes_sample.csv` (__ЛР2__) на предложения, а предложения - на слова (используйте `sent_tokenize` и `word_tokenize` из `nltk`). Каждый отзыв представьте в виде списка списков: внешний список - предложения, вложенные списки - слова в предложении. Исключите знаки препинания из списков слов.

`'Предложение номер один. Предложение номер два.' => [['Предложение', 'номер', 'один'], ['Предложение', 'номер', 'два']]`

In [26]:
import nltk
#nltk.download('punkt')
sentences_dict = {}
descriptions = recipes["name"].to_frame().join(recipes["description"].fillna(""))

for item in descriptions["description"].to_list():
    sentences_dict[item] = []
    
    for sentence in nltk.sent_tokenize(item):
        words_list = nltk.word_tokenize(sentence)
        sentences_dict[item].append(words_list)

3.2 Посчитайте кол-во уникальных слов среди всех отзывов (без учета регистра и знаков препинания).

In [ ]:
email = "try"
pattern = re.compile(r'^[a-zA-Z]+$')
result = pattern.match(email)
print(result)
if result is None:
    print("String is incorrect")
else:
    print("String is correct")


pattern = re.compile(r'(^(^[!"#$%&\'()*+\,\-.\/:;<=>?@[\]^_`{|}~]|^[0-9]))')
data = []
for sentences in sentences_dict.values():
    new_words = []
    [[new_words.append(y.lower()) for y in x if pattern.match(y) is None] for x in sentences]
    data.extend(new_words)

print(f"{len(data)} number of unique words {len(set(data))} ")

3.3 Найдите 5 самых длинных (по количеству слов; без учета знаков препинания) отзывов в датасете и выведите их в порядке убывания длины.

In [ ]:
new_list = []

for key, values in sentences_dict.items():
    
    counter = 0
    for item in values:
        new_words = [word for word in item if pattern.match(word) is None]
        counter += len(new_words)
        
    new_list.append((counter, key))
    
new_list.sort(key=lambda x: x[0],reverse=True)
    
for item in new_list[:5]:
    print(item)
    print("--------")

3.4 Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
``` 
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на любом предложении из отзывов.


In [31]:
import re
from typing import List
#nltk.download('averaged_perceptron_tagger')

class SentenceManager:
    PATTERN = re.compile(r'(^(^[!"#$%&\'()*+\,\-.\/:;<=>?@[\]^_`{|}~]|^[0-9]))')
    
    @validate_arguments
    def __init__(self, sentence : List[str]) -> None:
        self._sentence = sentence
        self._results = None
        self.processing()

    def processing(self) -> None:
        new_words = [word for word in self._sentence if self.PATTERN.match(word) is None]
        self._results = nltk.pos_tag(new_words)
    
    def info(self):
        str_results_first, str_results_second = [], []
        for item in self._results:
            word_len = max(map(len,item))+1
            
            buffer1 = "{0:^{word_len}}".format(item[1],word_len=word_len)
            str_results_first.append(buffer1)
            
            buffer2 = "{0:^{word_len}}".format(item[0], word_len=word_len)
            str_results_second.append(buffer2)

        result = f"{' '.join(str_results_first)}\n{' '.join(str_results_second)}\n"
        print(result)


review = random.choice(list(sentences_dict.values()))
for sentence in review:
    manager = SentenceManager(sentence)
    manager.info()

  NN       NN      VBZ  DT    NN      NN    
shrimp  cocktail  gets   a  latin  makeover 

 DT   VBZ   IN    DT     NN     NN   NN  
this   is  from  the  rachael  ray  show 

